In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import boxcox

In [2]:
test = pd.read_csv(r"C:\Users\longv\Downloads\house-prices-advanced-regression-techniques (1)\test.csv")

In [3]:
data = pd.read_csv(r"C:\Users\longv\Prediction-House-Price\Data\raw data\RawData.csv")
data = data.fillna(0)
data = data.drop(['SalePrice'], axis = 1)
numeric_features = data.dtypes[data.dtypes != "object"].index
data_num = data[numeric_features]
skewed_feats = data_num.apply(lambda x : skew(x)).sort_values(ascending  = True)
high_skew = skewed_feats[abs(skewed_feats) > 0.5]
skew_feat = high_skew.index
skew_feat


Index(['GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath',
       'Fireplaces', 'HalfBath', 'TotRmsAbvGrd', 'OverallCond', '2ndFlrSF',
       'BsmtUnfSF', 'GrLivArea', '1stFlrSF', 'MSSubClass', 'TotalBsmtSF',
       'WoodDeckSF', 'BsmtFinSF1', 'OpenPorchSF', 'MasVnrArea',
       'EnclosedPorch', 'BsmtHalfBath', 'ScreenPorch', 'BsmtFinSF2',
       'KitchenAbvGr', 'LowQualFinSF', '3SsnPorch', 'LotArea', 'PoolArea',
       'MiscVal'],
      dtype='object')

In [4]:

def Split_DF(df):
    numeric_features = df.dtypes[df.dtypes != "object"].index
    object_features = df.dtypes[df.dtypes == "object"].index
    return df[numeric_features], df[object_features]


In [5]:
#skew => boxcox tranformation ( for numeric data)
def Handle_Skew(df, list_feat_skew):
    for feat in list_feat_skew:
        pre = np.array(df[feat])
        pre = np.abs(pre + 1)
        tran_, _ = boxcox(pre)
        df[feat] = tran_
    return df 

In [6]:
# One hot encoding for object data

def one_hot_encoding(df):
    for col in df.columns:
        dummy_columns = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df, dummy_columns], axis=1)
        df.drop(columns=[col], inplace=True)
    col = df.columns
    df[col] = df[col].astype(int)
    return df

In [7]:
def Processing_Data(data, list_):
    # ----- Handle Y ---------#
    data = data.fillna(0)
    #---------Handle X (feature) boxcox transform and onehotencoding------------#
    numeric_data, object_data = Split_DF(data)
    numeric_data = Handle_Skew(numeric_data, list_)
    object_data = one_hot_encoding(object_data)
    data = pd.concat([numeric_data, object_data], axis = 1)

    #------Fill N/A = 0---------#
  

    # --------- Drop and add new feature -------#
    data = data.drop(['Id'], axis=1)
    data['TotalSF'] = (data['TotalBsmtSF'] 
                       + data['1stFlrSF'] 
                       + data['2ndFlrSF'])
    data['YrBltAndRemod'] = data['YearBuilt'] +data['YearRemodAdd']
    data['Total_sqr_footage'] = (data['BsmtFinSF1'] 
                                 + data['BsmtFinSF2'] 
                                 + data['1stFlrSF'] 
                                 + data['2ndFlrSF']
                                )
    data['Total_Bathrooms'] = (data['FullBath'] 
                               + (0.5 * data['HalfBath']) 
                               + data['BsmtFullBath'] 
                               + (0.5 * data['BsmtHalfBath'])
                              )
    data['Total_porch_sf'] = (data['OpenPorchSF'] 
                              + data['3SsnPorch'] 
                              + data['EnclosedPorch'] 
                              + data['ScreenPorch'] 
                              + data['WoodDeckSF']
                             )
    data['haspool'] = data['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    data['has2ndfloor'] = data['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    data['hasgarage'] = data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    data['hasbsmt'] = data['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    data['hasfireplace'] = data['Fireplaces'].apply(lambda x: 1 if x > 0 else 0) 
    return data 

In [8]:

new_test = Processing_Data(test, skew_feat)

C:\Users\longv\AppData\Local\Temp\ipykernel_24980\3422378078.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feat] = tran_


In [9]:
new_test

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalSF,YrBltAndRemod,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,2.194613,80.0,67.412710,5,4.284374,2.045257e+71,5.594016e+106,0.000000,13.153325,0.709988,...,241.242311,5.594016e+106,19.600975,1.000000,4.548280,0,0,1,1,0
1,2.194613,81.0,72.474179,6,4.284374,1.977352e+71,5.319522e+106,2.611418,16.024589,0.000000,...,328.908598,5.319522e+106,22.037119,1.171164,8.519270,0,0,1,1,0
2,2.667123,74.0,71.683468,5,3.684399,3.054824e+71,1.033968e+107,0.000000,15.333371,0.000000,...,254.935636,1.033968e+107,25.360320,2.171164,8.073713,0,1,1,1,1
3,2.667123,78.0,63.869567,6,4.284374,3.088739e+71,1.033968e+107,2.048813,14.167329,0.000000,...,254.514383,1.033968e+107,24.179602,2.171164,8.465880,0,1,1,1,1
4,2.915289,43.0,49.943360,8,3.684399,2.890514e+71,9.366599e+106,0.000000,11.036878,0.000000,...,319.683283,9.366599e+106,17.023452,2.000000,5.212116,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,3.007849,21.0,35.393595,4,4.861303,2.262587e+71,6.502451e+106,0.000000,0.000000,0.000000,...,172.271271,6.502451e+106,9.549037,1.171164,0.000000,0,1,0,1,0
1455,3.007849,21.0,35.109869,4,3.684399,2.262587e+71,6.502451e+106,0.000000,10.890620,0.000000,...,172.271271,6.502451e+106,20.439657,1.171164,3.376855,0,1,1,1,0
1456,2.194613,160.0,81.618376,5,4.861303,2.022378e+71,1.000490e+107,0.000000,17.351955,0.000000,...,309.037586,1.000490e+107,23.307555,1.430494,4.821587,0,0,1,1,1
1457,2.796419,62.0,64.904213,5,3.684399,2.890514e+71,9.366599e+106,0.000000,11.913920,0.000000,...,247.443735,9.366599e+106,17.707349,1.022369,7.363975,0,0,0,1,0


In [10]:
X_train = pd.read_csv(r"C:\Users\longv\Prediction-House-Price\Data\handled data\X_train.csv")
y_train = pd.read_csv(r"C:\Users\longv\Prediction-House-Price\Data\handled data\y_train.csv")
X_test = pd.read_csv(r"C:\Users\longv\Prediction-House-Price\Data\handled data\X_test.csv")
y_test = pd.read_csv(r"C:\Users\longv\Prediction-House-Price\Data\handled data\y_test.csv")

In [11]:
columns_A = X_train.columns

# Reindex DataFrame B để có tất cả các cột giống với A
new_test_= new_test.reindex(columns=columns_A, fill_value=0)

In [12]:
new_test_

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalSF,YrBltAndRemod,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,2.194613,80.0,67.412710,5,4.284374,2.045257e+71,5.594016e+106,0.000000,13.153325,0.709988,...,241.242311,5.594016e+106,19.600975,1.000000,4.548280,0,0,1,1,0
1,2.194613,81.0,72.474179,6,4.284374,1.977352e+71,5.319522e+106,2.611418,16.024589,0.000000,...,328.908598,5.319522e+106,22.037119,1.171164,8.519270,0,0,1,1,0
2,2.667123,74.0,71.683468,5,3.684399,3.054824e+71,1.033968e+107,0.000000,15.333371,0.000000,...,254.935636,1.033968e+107,25.360320,2.171164,8.073713,0,1,1,1,1
3,2.667123,78.0,63.869567,6,4.284374,3.088739e+71,1.033968e+107,2.048813,14.167329,0.000000,...,254.514383,1.033968e+107,24.179602,2.171164,8.465880,0,1,1,1,1
4,2.915289,43.0,49.943360,8,3.684399,2.890514e+71,9.366599e+106,0.000000,11.036878,0.000000,...,319.683283,9.366599e+106,17.023452,2.000000,5.212116,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,3.007849,21.0,35.393595,4,4.861303,2.262587e+71,6.502451e+106,0.000000,0.000000,0.000000,...,172.271271,6.502451e+106,9.549037,1.171164,0.000000,0,1,0,1,0
1455,3.007849,21.0,35.109869,4,3.684399,2.262587e+71,6.502451e+106,0.000000,10.890620,0.000000,...,172.271271,6.502451e+106,20.439657,1.171164,3.376855,0,1,1,1,0
1456,2.194613,160.0,81.618376,5,4.861303,2.022378e+71,1.000490e+107,0.000000,17.351955,0.000000,...,309.037586,1.000490e+107,23.307555,1.430494,4.821587,0,0,1,1,1
1457,2.796419,62.0,64.904213,5,3.684399,2.890514e+71,9.366599e+106,0.000000,11.913920,0.000000,...,247.443735,9.366599e+106,17.707349,1.022369,7.363975,0,0,0,1,0


In [15]:
new_test_.to_csv(r"C:\Users\longv\Prediction-House-Price\Data\handled data\unlabeleddata.csv", index = False)

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import Lasso , Ridge, ElasticNet, BayesianRidge
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [15]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
def crossValidModel(model, k):
    cv =  KFold(shuffle=True, random_state=2, n_splits=k)
    return -1*cross_val_score(model , x_train ,y_train,cv = cv, scoring = 'neg_mean_absolute_error').mean()

In [16]:
ridge = Ridge(alpha= 0.01)
    ## fit the model. 
ridge.fit(X_train, y_train)

Ridge(alpha=0.01)

In [17]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalSF,YrBltAndRemod,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,2.191299,90.0,10.858967,9,2.235408,5.454641e+71,2.789399e+132,452.0,6.276152,0.0,...,353.183960,2.789399e+132,11.924675,2.000000,5.232535,0,0,1,1,1
1,2.191299,60.0,10.102310,5,2.235408,3.303366e+71,1.111687e+132,0.0,0.000000,0.0,...,210.039029,1.111687e+132,5.244874,1.000000,0.000000,0,0,1,1,0
2,2.376658,80.0,11.037080,5,2.691919,2.070696e+71,2.733569e+132,0.0,15.279601,0.0,...,207.008198,2.733569e+132,20.568100,1.425418,0.701086,0,0,1,1,0
3,2.191299,0.0,11.027518,8,2.235408,5.161875e+71,2.520944e+132,148.0,17.837168,0.0,...,316.494144,2.520944e+132,23.442175,2.425418,8.339691,0,0,1,1,1
4,2.191299,118.0,11.071160,7,2.235408,5.105151e+71,2.520944e+132,150.0,0.000000,0.0,...,312.460363,2.520944e+132,5.554205,2.000000,9.398877,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,2.661928,82.0,10.572753,8,2.235408,4.993483e+71,2.372150e+132,673.0,17.603874,0.0,...,273.119092,2.372150e+132,27.680831,2.601479,5.410631,0,1,1,1,1
1164,2.191299,60.0,10.596445,4,2.691919,2.884236e+71,2.187017e+132,0.0,13.247122,0.0,...,238.924778,2.187017e+132,18.593958,2.000000,4.577236,0,0,1,1,0
1165,2.810822,68.0,10.500590,6,2.235408,3.953365e+71,1.545527e+132,0.0,0.000000,0.0,...,9.830102,1.545527e+132,9.830102,2.000000,0.000000,0,1,1,0,0
1166,2.908859,0.0,9.162122,7,2.235408,5.219201e+71,2.625158e+132,18.0,0.000000,0.0,...,287.409712,2.625158e+132,5.559946,2.000000,7.213255,0,0,1,1,1


In [18]:
new_test_

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalSF,YrBltAndRemod,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,2.194613,80.0,67.412710,5,4.284374,2.045257e+71,5.594016e+106,0.000000,13.153325,0.709988,...,241.242311,5.594016e+106,19.600975,1.000000,4.548280,0,0,1,1,0
1,2.194613,81.0,72.474179,6,4.284374,1.977352e+71,5.319522e+106,2.611418,16.024589,0.000000,...,328.908598,5.319522e+106,22.037119,1.171164,8.519270,0,0,1,1,0
2,2.667123,74.0,71.683468,5,3.684399,3.054824e+71,1.033968e+107,0.000000,15.333371,0.000000,...,254.935636,1.033968e+107,25.360320,2.171164,8.073713,0,1,1,1,1
3,2.667123,78.0,63.869567,6,4.284374,3.088739e+71,1.033968e+107,2.048813,14.167329,0.000000,...,254.514383,1.033968e+107,24.179602,2.171164,8.465880,0,1,1,1,1
4,2.915289,43.0,49.943360,8,3.684399,2.890514e+71,9.366599e+106,0.000000,11.036878,0.000000,...,319.683283,9.366599e+106,17.023452,2.000000,5.212116,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,3.007849,21.0,35.393595,4,4.861303,2.262587e+71,6.502451e+106,0.000000,0.000000,0.000000,...,172.271271,6.502451e+106,9.549037,1.171164,0.000000,0,1,0,1,0
1455,3.007849,21.0,35.109869,4,3.684399,2.262587e+71,6.502451e+106,0.000000,10.890620,0.000000,...,172.271271,6.502451e+106,20.439657,1.171164,3.376855,0,1,1,1,0
1456,2.194613,160.0,81.618376,5,4.861303,2.022378e+71,1.000490e+107,0.000000,17.351955,0.000000,...,309.037586,1.000490e+107,23.307555,1.430494,4.821587,0,0,1,1,1
1457,2.796419,62.0,64.904213,5,3.684399,2.890514e+71,9.366599e+106,0.000000,11.913920,0.000000,...,247.443735,9.366599e+106,17.707349,1.022369,7.363975,0,0,0,1,0


In [19]:
from sklearn.preprocessing import StandardScaler

# Assuming data is your dataset
# data should be a 2-dimensional array or a pandas DataFrame
# Select only the numerical columns for scaling
numeric_columns = X_train.select_dtypes(include=['int', 'float']).columns

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the data
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])

In [20]:
from sklearn.preprocessing import StandardScaler

# Assuming data is your dataset
# data should be a 2-dimensional array or a pandas DataFrame
# Select only the numerical columns for scaling
numeric_columns = X_test.select_dtypes(include=['int', 'float']).columns

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the data
X_test[numeric_columns] = scaler.fit_transform(X_test[numeric_columns])

In [21]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalSF,YrBltAndRemod,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,-1.160814,0.973557,0.484132,2.131506,-0.471259,1.358693,1.220290,1.914933,-0.458460,-0.364528,...,1.713235,1.220290,-0.713849,0.319333,0.221271,-0.058621,-0.880384,0.228588,0.156721,0.945003
1,-1.160814,0.089079,-0.618600,-0.794852,-0.471259,-0.456405,-1.134508,-0.564793,-1.341610,-0.364528,...,-0.355182,-1.134508,-1.666739,-1.372456,-1.405762,-0.058621,-0.880384,0.228588,0.156721,-1.058198
2,-0.506353,0.678731,0.743710,-0.794852,1.258337,-1.496447,1.141927,-0.564793,0.808461,-0.364528,...,-0.398977,1.141927,0.519155,-0.652739,-1.187763,-0.058621,-0.880384,0.228588,0.156721,-1.058198
3,-1.160814,-1.679877,0.729775,1.399917,-0.471259,1.111677,0.843493,0.247153,1.168350,-0.364528,...,1.183074,0.843493,0.929149,1.039050,1.187426,-0.058621,-0.880384,0.228588,0.156721,0.945003
4,-1.160814,1.799069,0.793378,0.668327,-0.471259,1.063817,0.843493,0.258125,-1.341610,-0.364528,...,1.124787,0.843493,-1.622612,0.319333,1.516776,-0.058621,-0.880384,0.228588,0.156721,0.945003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,0.500865,0.737696,0.067011,1.399917,-0.471259,0.969599,0.634648,3.127366,1.135522,-0.364528,...,0.556313,0.634648,1.533803,1.336908,0.276649,-0.058621,1.135868,0.228588,0.156721,0.945003
1164,-1.160814,0.089079,0.101540,-1.526442,1.258337,-0.810038,0.374800,-0.564793,0.522461,-0.364528,...,0.062212,0.374800,0.237539,0.319333,0.017508,-0.058621,-0.880384,0.228588,0.156721,-1.058198
1165,1.026575,0.324940,-0.038157,-0.063262,-0.471259,0.092019,-0.525580,-0.564793,-1.341610,-0.364528,...,-3.248162,-0.525580,-1.012645,0.319333,-1.405762,-0.058621,1.135868,0.228588,-6.380775,-1.058198
1166,1.372722,-1.679877,-1.988807,0.668327,-0.471259,1.160044,0.989764,-0.466043,-1.341610,-0.364528,...,0.762810,0.989764,-1.621793,0.319333,0.837167,-0.058621,-0.880384,0.228588,0.156721,0.945003


In [22]:
from sklearn.preprocessing import StandardScaler

# Assuming data is your dataset
# data should be a 2-dimensional array or a pandas DataFrame
# Select only the numerical columns for scaling
numeric_columns = new_test_.select_dtypes(include=['int', 'float']).columns

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the data
new_test_[numeric_columns] = scaler.fit_transform(new_test_[numeric_columns])

In [23]:
new_test_

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalSF,YrBltAndRemod,Total_sqr_footage,Total_Bathrooms,Total_porch_sf,haspool,has2ndfloor,hasgarage,hasbsmt,hasfireplace
0,-1.153867,0.684849,0.528189,-0.751101,0.420405,-0.501416,-1.100892,-0.792269,0.563679,2.666358,...,-0.331806,-1.100892,0.371866,-1.335202,-0.044196,-0.06426,-0.859637,0.236043,0.172163,-0.999315
1,-1.153867,0.715852,1.012427,-0.054877,0.420405,-0.592669,-1.200419,1.163005,0.982248,-0.375141,...,0.645628,-1.200419,0.729100,-1.055438,1.192988,-0.06426,-0.859637,0.236043,0.172163,-0.999315
2,0.484391,0.498831,0.936778,-0.751101,-0.479861,0.855278,0.619800,-0.792269,0.881483,-0.375141,...,-0.179133,0.619800,1.216412,0.579039,1.054172,-0.06426,1.163282,0.236043,0.172163,1.000686
3,0.484391,0.622843,0.189212,-0.054877,0.420405,0.900854,0.619800,0.741760,0.711499,-0.375141,...,-0.183830,0.619800,1.043272,0.579039,1.176354,-0.06426,1.163282,0.236043,0.172163,1.000686
4,1.344816,-0.462261,-1.143126,1.337571,-0.479861,0.634472,0.266980,-0.792269,0.255146,-0.375141,...,0.542770,0.266980,-0.006100,0.299275,0.162625,-0.06426,-0.859637,0.236043,0.172163,-0.999315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1.665737,-1.144325,-2.535121,-1.447325,1.286091,-0.209359,-0.771509,-0.792269,-1.353795,-0.375141,...,-1.100798,-0.771509,-1.102142,-1.055438,-1.461239,-0.06426,1.163282,-4.236514,0.172163,-0.999315
1455,1.665737,-1.144325,-2.562266,-1.447325,-0.479861,-0.209359,-0.771509,-0.792269,0.233825,-0.375141,...,-1.100798,-0.771509,0.494850,-1.055438,-0.409161,-0.06426,1.163282,0.236043,0.172163,-0.999315
1456,-1.153867,3.165085,1.887264,-0.751101,1.286091,-0.532161,0.498418,-0.792269,1.175749,-0.375141,...,0.424076,0.498418,0.915396,-0.631569,0.040954,-0.06426,-0.859637,0.236043,0.172163,1.000686
1457,0.932679,0.126795,0.288198,-0.751101,-0.479861,0.634472,0.266980,-0.792269,0.383000,-0.375141,...,-0.262664,0.266980,0.094186,-1.298640,0.833049,-0.06426,-0.859637,-4.236514,0.172163,-0.999315


In [24]:
ridge = Ridge(alpha= 0.01)
    ## fit the model. 
ridge.fit(X_train, y_train)

Ridge(alpha=0.01)

In [81]:
from scipy.stats import uniform, randint
param_grid = {
    'n_estimators': [100,200,300],  # Number of boosting stages
    'max_depth': [4,5,6],       # Maximum depth of the individual estimators
    'learning_rate':[0.005, 0.05, 0.0005],# Learning rate
    'min_samples_split': [4,7,9],# Minimum number of samples required to split a node
    'min_samples_leaf':[5,8,10,12] # Minimum number of samples required at each leaf node
}
model = GradientBoostingRegressor()
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Best Parameters: ", random_search.best_params_)
print("Best Score: ", -random_search.best_score_)                                                  

Best Parameters:  {'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_depth': 4, 'learning_rate': 0.05}
Best Score:  0.017377475321803806


In [119]:
best_model = GradientBoostingRegressor(n_estimators = 320, min_samples_split = 4, min_samples_leaf = 5, max_depth = 4, learning_rate = 0.0612)
best_model.fit(X_train, y_train)
pred = best_model.predict(X_test)
print( "test_loss = ", mean_absolute_error(pred, y_test))

test_loss =  0.08264944069115449


In [182]:
best_model_ = HistGradientBoostingRegressor(loss='squared_error', quantile=True, learning_rate=0.05, max_iter=450, 
                                            max_leaf_nodes=31, max_depth=5, min_samples_leaf=20, l2_regularization=0.0, max_features=1.0, 
                                            max_bins=255, categorical_features='warn', monotonic_cst=None, interaction_cst=None, warm_start=True,
                                            early_stopping=False, scoring='loss', 
                                            validation_fraction=0.1, n_iter_no_change=50, tol=1e-07, verbose=0, random_state=None)
best_model_.fit(X_train, y_train)
pred = best_model_.predict(X_test)
print( "test_loss = ", mean_absolute_error(pred, y_test))

test_loss =  0.08331296297318129


In [73]:
lrs = [ 0.07, 0.09]
mds = [6,7,8]
for lr in lrs:
    for md in mds:
        gbr = GradientBoostingRegressor(loss='squared_ro', learning_rate = lr, n_estimators=100, subsample=1.0, criterion='squared_error',
                                        min_samples_split=10, min_samples_leaf=7, min_weight_fraction_leaf=0, max_depth=md, min_impurity_decrease=0.0, init=None, random_state=5,
                                        max_features=None, alpha=0.01, verbose=0, max_leaf_nodes=12, warm_start=True, validation_fraction=0.1, n_iter_no_change=None, 
                                        tol=0.001, ccp_alpha=0.0).fit(X_train, y_train)
        y_pred_grb = gbr.predict(X_test)
        print("with lr = ", lr)
        print(" with maxdepht = ", md)
        print( "test_loss = ", mean_absolute_error(y_pred_grb, y_test))
        
        print("trainloss = ", mean_absolute_error(y_train, gbr.predict(X_train)))

with lr =  0.07
 with maxdepht =  6
test_loss =  0.08960198533123256
trainloss =  0.05209770915703315
with lr =  0.07
 with maxdepht =  7
test_loss =  0.0895600798971521
trainloss =  0.051457312977430725
with lr =  0.07
 with maxdepht =  8
test_loss =  0.08866133062790024
trainloss =  0.05136196016245936
with lr =  0.09
 with maxdepht =  6
test_loss =  0.08970741498008973
trainloss =  0.046591522901220095
with lr =  0.09
 with maxdepht =  7
test_loss =  0.09049884806840636
trainloss =  0.04631946642029056
with lr =  0.09
 with maxdepht =  8
test_loss =  0.08938363972990303
trainloss =  0.045865630184594455


In [31]:
BR = BayesianRidge()
BR.fit(X_train, y_train)

BayesianRidge()

In [32]:
y_pred_grb = BR.predict(X_test)

print( "test_loss = ", mean_absolute_error(y_pred_grb, y_test))
        
print("trainloss = ", mean_absolute_error(y_train, gbr.predict(X_train)))

test_loss =  0.09202852143687336
trainloss =  0.018349266690204712


In [57]:
gbr =GradientBoostingRegressor(loss='squared_error', learning_rate=0.06, n_estimators=475, subsample=1.0, criterion='squared_error',
                                        min_samples_split=7, min_samples_leaf=7, min_weight_fraction_leaf=0.0, max_depth=5, min_impurity_decrease=0.0, init=None, random_state=None,
                                        max_features=None, alpha=0.01, verbose=0, max_leaf_nodes=15, warm_start=True, validation_fraction=0.1, n_iter_no_change=None, 
                                        tol=0.001, ccp_alpha=0.0).fit(X_train, y_train)

KeyboardInterrupt: 

In [174]:
from sklearn.linear_model import ElasticNet
alpha_EN = [ 0.005, 0.007, 0.009 ]
for alpha in alpha_EN:
    EN_model = ElasticNet(alpha = alpha)
    EN_model.fit(X_train, y_train)
    y_pred = EN_model.predict(X_test)
    print("with alpha = ", alpha)
       
    print( "test_loss = ", mean_absolute_error(y_pred, y_test))
        
    print("trainloss = ", mean_absolute_error(y_train, EN_model.predict(X_train)))

with alpha =  0.005
test_loss =  0.08630703118935912
trainloss =  0.06493741525947166
with alpha =  0.007
test_loss =  0.08603536672398512
trainloss =  0.06662708491092223
with alpha =  0.009
test_loss =  0.0862294028987809
trainloss =  0.06816127386793606


In [178]:
EN_model = ElasticNet(alpha = 0.07)
EN_model.fit(X_train, y_train)

ElasticNet(alpha=0.07)

In [53]:
pred = gbr.predict(new_test_)

In [54]:
pred = np.exp(pred)

In [55]:
pred

array([117777.21892223, 160143.23773393, 191547.31278602, ...,
       163860.04913842, 112685.46939083, 233192.723096  ])

In [56]:
df_test = pd.read_csv(r"C:\Users\longv\Downloads\house-prices-advanced-regression-techniques\test.csv")
df_test['SalePrice'] = pred
df_test[['Id','SalePrice']].to_csv(r"C:\Users\longv\predict.csv", index = False)